In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from numpy.linalg import norm

# 1. Load data
df_prodi = pd.read_csv('daftar_prodi.csv')
df_univ = pd.read_csv('daftar_universitas.csv')

# Merge nama_univ ke df_prodi supaya mudah ditampilkan
df = df_prodi.merge(df_univ[['kode_univ', 'nama_univ']], on='kode_univ', how='left')

# 2. Fitur dan target untuk model ML
fitur = ['peminat_2018', 'peminat_2019', 'peminat_2020', 'peminat_2021', 'peminat_2022',
         'daya_tampung_2018', 'daya_tampung_2019', 'daya_tampung_2020', 'daya_tampung_2021', 'daya_tampung_2022']
df = df.dropna(subset=fitur)

# Target: rasio peluang keterimaan tahun terakhir (2022)
df['peluang_2022'] = df['daya_tampung_2022'] / df['peminat_2022'].replace(0, np.nan)
df['peluang_2022'] = df['peluang_2022'].fillna(0)

X = df[fitur].values
y = df['peluang_2022'].values

# 3. Latih model Linear Regression
model = LinearRegression()
model.fit(X, y)

# 4. Prediksi peluang keterimaan dengan model
df['prediksi_peluang'] = model.predict(X)

# 5. Normalisasi hasil prediksi
scaler = MinMaxScaler()
df['prediksi_scaled'] = scaler.fit_transform(df[['prediksi_peluang']])

# 6a. Rekomendasi berdasarkan nama prodi
def rekomendasi_berdasarkan_prodi(nama_prodi_input, top_n=10):
    mask = df['nama_prodi'].str.lower().str.contains(nama_prodi_input.lower())
    if not mask.any():
        return f"Program studi '{nama_prodi_input}' tidak ditemukan."

    subdf = df[mask].copy()
    subdf_sorted = subdf.sort_values(by='prediksi_scaled', ascending=False).head(top_n)

    rekom_list = []
    for i, row in subdf_sorted.iterrows():
        rekom_list.append({
            'rank': len(rekom_list) + 1,
            'nama_prodi': row['nama_prodi'],
            'nama_univ': row['nama_univ'],
            'prediksi_peluang': row['prediksi_peluang'],
            'prediksi_scaled': row['prediksi_scaled']
        })
    return rekom_list

# 6b. Rekomendasi berdasarkan nama universitas
def rekomendasi_berdasarkan_universitas(nama_univ_input, top_n=10):
    mask = df['nama_univ'].str.lower().str.contains(nama_univ_input.lower())
    if not mask.any():
        return f"Universitas '{nama_univ_input}' tidak ditemukan."

    subdf = df[mask].copy()
    subdf_sorted = subdf.sort_values(by='prediksi_scaled', ascending=False).head(top_n)

    rekom_list = []
    for i, row in subdf_sorted.iterrows():
        rekom_list.append({
            'rank': len(rekom_list) + 1,
            'nama_prodi': row['nama_prodi'],
            'nama_univ': row['nama_univ'],
            'prediksi_peluang': row['prediksi_peluang'],
            'prediksi_scaled': row['prediksi_scaled']
        })
    return rekom_list

# 7. Interaktif input dari user
print("=== Rekomendasi Program Studi Berdasarkan Prediksi Peluang Keterimaan (AI Model) ===")
print("Apakah kamu ingin rekomendasi berdasarkan:")
print("1. Nama Program Studi (Jurusan)")
print("2. Nama Universitas")
pilihan = input("Masukkan pilihan (1 atau 2): ").strip()

if pilihan == "1":
    input_prodi = input("Masukkan nama program studi: ").strip()
    hasil = rekomendasi_berdasarkan_prodi(input_prodi, top_n=10)
    if isinstance(hasil, str):
        print(hasil)
    else:
        print(f"\nRekomendasi program studi '{input_prodi}' dari berbagai universitas berdasarkan prediksi AI:\n")
        for r in hasil:
            print(f"{r['rank']}. {r['nama_prodi']} ({r['nama_univ']})")
            print(f"   Prediksi Peluang: {r['prediksi_peluang']:.3f} | Skala: {r['prediksi_scaled']:.3f}")

elif pilihan == "2":
    input_univ = input("Masukkan nama universitas: ").strip()
    hasil = rekomendasi_berdasarkan_universitas(input_univ, top_n=10)
    if isinstance(hasil, str):
        print(hasil)
    else:
        print(f"\nRekomendasi program studi dari '{input_univ}' berdasarkan prediksi AI:\n")
        for r in hasil:
            print(f"{r['rank']}. {r['nama_prodi']} ({r['nama_univ']})")
            print(f"   Prediksi Peluang: {r['prediksi_peluang']:.3f} | Skala: {r['prediksi_scaled']:.3f}")
else:
    print("Pilihan tidak valid. Harap masukkan 1 atau 2.")

=== Rekomendasi Program Studi Berdasarkan Prediksi Peluang Keterimaan (AI Model) ===
Apakah kamu ingin rekomendasi berdasarkan:
1. Nama Program Studi (Jurusan)
2. Nama Universitas

Rekomendasi program studi 'Ilmu Komputer' dari berbagai universitas berdasarkan prediksi AI:

1. ILMU KOMPUTER (UNIVERSITAS SEMBILAN BELAS NOVEMBER KOLAKA)
   Prediksi Peluang: 0.706 | Skala: 0.794
2. ILMU KOMPUTER (UNIVERSITAS NUSA CENDANA)
   Prediksi Peluang: 0.539 | Skala: 0.740
3. ILMU KOMPUTER (UNIVERSITAS NEGERI JAKARTA)
   Prediksi Peluang: 0.500 | Skala: 0.728
4. ILMU KOMPUTER (UNIVERSITAS HALUOLEO)
   Prediksi Peluang: 0.498 | Skala: 0.727
5. PENDIDIKAN ILMU KOMPUTER (UNIVERSITAS PENDIDIKAN INDONESIA)
   Prediksi Peluang: 0.456 | Skala: 0.713
6. ILMU KOMPUTER (UNIVERSITAS LAMBUNG MANGKURAT)
   Prediksi Peluang: 0.379 | Skala: 0.688
7. ILMU KOMPUTER (UNIVERSITAS LAMPUNG)
   Prediksi Peluang: 0.286 | Skala: 0.658
8. ILMU KOMPUTER (UNIVERSITAS ISLAM NEGERI SUMATERA UTARA)
   Prediksi Peluang: 0.237 | 